Нестеров Владислав, 569МН група, Варіант 8

Частота f[МГц] = 560
Число елементів антени N = 9

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from scipy.constants import c as light_speed

In [4]:
FC = [1] # "множник грат" : ДС антени в площині Н
FE = [1] # ДС антени в площині Е
F1E = [1] # ДС напівхвильового вібратора в площині Е

steps = [0] # кути θ
SGP1 = 0 # Ширина головної пелюстки в площині FH
SGP2 = 0 # Ширина головної пелюстки в площині FE

Zeros = [] # нульові кути

In [5]:
N = 9 # Кількість елементів антени
F = 560e6 # Частота переведена в Гц

wavelength = round(light_speed / F, 4)
dcp = round(0.25 * wavelength, 4) 
k = round((2 * np.pi) / wavelength, 4) 

print(f"λ = {wavelength} m")
print(f"dcp = {dcp} m")
print(f"k = {k} rad/m")

λ = 0.5353 m
dcp = 0.1338 m
k = 11.7377 rad/m
